In [ ]:
import numpy as np
from scipy.spatial import KDTree
from astropy.io import fits
from astropy.table import QTable
from astropy import units as u
from astropy.coordinates import SkyCoord
import pylab as plt

In [ ]:
datafd = fits.open("../data/gaia_G20.0.fits") # note terrible file path
data_table = datafd[1].data
print(len(data_table))

In [ ]:
randfd = fits.open("../data/random_stardustm1064_G20.0_10x.fits") # note terrible file path
rand_table = QTable(randfd[1].data)
print(len(rand_table))

In [ ]:
# cut everything by the dust map
ebvlim = 0.05
Id = data_table['ebv'] < ebvlim
data_table = data_table[Id]
Ir = rand_table['ebv'] < ebvlim
rand_table = rand_table[Ir]

In [ ]:
plt.scatter(data_table['ra'], data_table['dec'], marker=".", s=0.1, alpha=0.1)
plt.figure()
plt.scatter(rand_table['ra'], rand_table['dec'], marker=".", s=0.1, alpha=0.1)

In [ ]:
# Add l, b to the random catalog.
# WARNING: This might be inconsistent with how l, b are calculated in the data table.
randc = SkyCoord(ra=rand_table['ra']*u.degree, dec=rand_table['dec']*u.degree, frame='icrs')
rand_table['l'] = randc.galactic.l.to(u.deg).value
rand_table['b'] = randc.galactic.b.to(u.deg).value

In [ ]:
dhi, dlo = data_table['b'] > 0., data_table['b'] < 0.
for I in [dhi, dlo]:
    plt.scatter(data_table['l'][I], np.sin(data_table['b'][I] * np.pi / 180.),
                marker=".", s=0.1, alpha=0.1)
plt.figure()
rhi, rlo = rand_table['b'] > 0., rand_table['b'] < 0.
for I in [rhi, rlo]:
    plt.scatter(rand_table['l'][I], np.sin(rand_table['b'][I] * np.pi / 180.),
                marker=".", s=0.1, alpha=0.1)

In [ ]:
factor = len(data_table) / len(rand_table)
bins = np.arange(0., 4.501, 0.1)
bincntrs = 0.5 * (bins[1:] + bins[:-1])
binwids = (bins[1:] - bins[:-1])
nzhi, _ = np.histogram(data_table['redshift_spz'][dhi], bins)
nzhi = nzhi.astype(float) / (np.sum(rhi) * factor) / binwids
nzlo, _ = np.histogram(data_table['redshift_spz'][dlo], bins)
nzlo = nzlo.astype(float) / (np.sum(rlo) * factor) / binwids

In [ ]:
plt.plot(bincntrs, nzhi, "o")
plt.plot(bincntrs, nzlo, "o")